In [289]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [290]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_id = test['id']

In [291]:
test = test.drop(['id'] , axis=1)


In [292]:
test

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Age at enrollment,International,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,1,1,1,9500,1,1,141.0,1,3,1,2,3,152.1,0,0,0,0,0,0,18,0,0,7,0,0,0.000000,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,1,1,1,9238,1,1,128.0,1,1,19,4,5,116.5,0,0,0,1,0,0,19,0,0,6,7,6,14.857143,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,1,1,1,9238,1,1,118.0,1,1,19,4,9,114.2,0,0,0,1,0,1,18,0,0,6,11,6,12.000000,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,1,44,1,9147,1,39,130.0,1,1,19,3,3,130.0,0,0,0,1,0,1,23,0,2,6,15,5,11.500000,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,1,39,1,9670,1,1,110.0,1,1,37,5,5,106.0,1,0,0,1,0,0,26,0,0,6,9,3,11.000000,0,0,6,9,4,10.666667,2,7.6,2.6,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,1,1,2,171,1,1,128.0,1,38,37,7,10,124.7,1,0,0,1,0,0,19,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51008,2,39,1,9119,1,19,133.1,1,19,37,9,9,140.0,0,0,1,0,1,0,33,0,0,5,6,0,0.000000,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12
51009,1,1,1,171,1,1,127.0,1,1,1,4,10,120.4,0,0,1,0,0,0,20,0,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51010,1,1,3,9773,1,1,132.0,1,19,19,5,5,126.3,1,0,1,0,0,0,18,0,0,6,8,5,12.600000,0,0,6,9,3,13.000000,0,7.6,2.6,0.32


In [293]:
data.columns


Index(['id', 'Marital status', 'Application mode', 'Application order',
       'Course', 'Daytime/evening attendance', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units

In [294]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def data_preprocessing (data):
    data = data.drop(columns = ["id"])
    data["Target"] = le.fit_transform(data["Target"])
    return data

data = data_preprocessing(data)


In [295]:
from sklearn.model_selection import train_test_split
X = data.drop(columns = ["Target"])
y = data["Target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [296]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler , RobustScaler
scaler =RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [297]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel

#model = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)
from sklearn.model_selection import GridSearchCV , RandomizedSearchCV
from xgboost import XGBClassifier


# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Initialize the XGBoost classifier
#xgb_model = AdaBoostClassifier(random_state=42)

# Initialize Grid Search
#grid_search = RandomizedSearchCV(estimator=xgb_model,param_distributions=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit the model
#grid_search.fit(X_train, y_train)

# Get the best parameters
#best_params = grid_search.best_params_
#print("Best parameters found: ", best_params)

# Train the model with the best parameters
# model = XGBClassifier(
#     n_estimators=best_params['n_estimators'],
#     max_depth=best_params['max_depth'],
#     learning_rate=best_params['learning_rate'],
#     subsample=best_params['subsample'],
#     colsample_bytree=best_params['colsample_bytree'],
#     random_state=42
# )
model = AdaBoostClassifier(n_estimators=200, random_state=42 , algorithm='SAMME.R', learning_rate=1)

model.fit(X_train , y_train)

/home/khadidja/.local/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=1, n_estimators=200, random_state=42)

In [298]:
model.score(X_test , y_test)

0.8288029273392578

In [299]:
test = scaler.transform(test)

In [300]:
y_predicted = model.predict(test)
y_predicted

array([0, 2, 2, ..., 0, 0, 0])

In [301]:
y_predicted = le.inverse_transform(y_predicted)

In [302]:
y_predicted[000:700]

array(['Dropout', 'Graduate', 'Graduate', 'Dropout', 'Enrolled',
       'Graduate', 'Graduate', 'Graduate', 'Dropout', 'Graduate',
       'Graduate', 'Graduate', 'Graduate', 'Enrolled', 'Dropout',
       'Enrolled', 'Graduate', 'Graduate', 'Enrolled', 'Graduate',
       'Graduate', 'Dropout', 'Graduate', 'Enrolled', 'Dropout',
       'Dropout', 'Dropout', 'Graduate', 'Graduate', 'Enrolled',
       'Graduate', 'Graduate', 'Dropout', 'Enrolled', 'Graduate',
       'Dropout', 'Dropout', 'Graduate', 'Graduate', 'Graduate',
       'Graduate', 'Graduate', 'Graduate', 'Graduate', 'Enrolled',
       'Graduate', 'Dropout', 'Dropout', 'Enrolled', 'Graduate',
       'Graduate', 'Graduate', 'Dropout', 'Dropout', 'Graduate',
       'Enrolled', 'Enrolled', 'Enrolled', 'Graduate', 'Graduate',
       'Enrolled', 'Graduate', 'Dropout', 'Graduate', 'Dropout',
       'Graduate', 'Enrolled', 'Dropout', 'Graduate', 'Enrolled',
       'Enrolled', 'Graduate', 'Graduate', 'Graduate', 'Dropout',
       'Enroll

In [303]:
submission = pd.DataFrame({"id":test_id , "Target": y_predicted})

In [304]:
submission.to_csv("submission12.csv" , index=False)